<a href="https://colab.research.google.com/github/manojmanivannan/ApacheSparkEssentials/blob/master/SparkByPluralsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install tools

In [1]:
# !apt-get update
# !apt-get install -y openjdk-8-jdk-headless scala  > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
# !tar xf spark-2.3.1-bin-hadoop2.7.tgz
# !pip install -q pyspark
# !pip install -q findspark
# !pip install -q yfinance

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,307 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://

In [2]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.master('local[2]').getOrCreate()
sc = spark.sparkContext

In [4]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

# Download data from yahoo finance

In [5]:
import yfinance as yf
stock_list = ['AAPL']
data = yf.download(stock_list, start="2021-01-01", end="2022-01-01")
data.to_csv('apple_stock.csv')

[*********************100%***********************]  1 of 1 completed


In [6]:
apple = sc.textFile('apple_stock.csv')
apple

apple_stock.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [7]:
apple.take(5)

['Date,Open,High,Low,Close,Adj Close,Volume',
 '2021-01-04,133.52000427246094,133.61000061035156,126.76000213623047,129.41000366210938,127.87493896484375,143301900',
 '2021-01-05,128.88999938964844,131.74000549316406,128.42999267578125,131.00999450683594,129.45596313476562,97664900',
 '2021-01-06,127.72000122070312,131.0500030517578,126.37999725341797,126.5999984741211,125.09827423095703,155088000',
 '2021-01-07,128.36000061035156,131.6300048828125,127.86000061035156,130.9199981689453,129.36703491210938,109578200']

In [8]:
apple.count()

253

In [9]:
## Download data from https://drive.google.com/drive/folders/0BzquUESTxeglWElscTF0Q19UTUU?resourcekey=0-ixxx-0VdlOFfz915kofCxQ


In [11]:
data = sc.textFile('nypd_crime.csv')

In [12]:
data.take(5)

['OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1',
 '1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"',
 '2,c6245d4d,12/14/1968 12:20:00 AM,Saturday,Dec,14,1968,0,12,14,2008,GRAND LARCENY,FELONY,G,28,MANHATTAN,N.Y. POLICE DEPT,996470,232106,"(40.8037530600001, -73.955861904)"',
 '3,716dbc6f,10/30/1970 03:30:00 PM,Friday,Oct,30,1970,15,10,31,2008,BURGLARY,FELONY,H,84,BROOKLYN,N.Y. POLICE DEPT,986508,190249,"(40.688874254, -73.9918594329999)"',
 '4,638cd7b7,07/18/1972 11:00:00 PM,Tuesday,Jul,18,1972,23,7,19,2012,GRAND LARCENY OF MOTOR VEHICLE,FELONY,F,73,BROOKLYN,N.Y. POLICE DEPT,1005876,182440,"(40.6674141890001, -73.9220463899999)"']

In [16]:
header = data.first()
dataWoHeader = data.filter(lambda x: x != header)

In [17]:
dataWoHeader.take(5)

['1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"',
 '2,c6245d4d,12/14/1968 12:20:00 AM,Saturday,Dec,14,1968,0,12,14,2008,GRAND LARCENY,FELONY,G,28,MANHATTAN,N.Y. POLICE DEPT,996470,232106,"(40.8037530600001, -73.955861904)"',
 '3,716dbc6f,10/30/1970 03:30:00 PM,Friday,Oct,30,1970,15,10,31,2008,BURGLARY,FELONY,H,84,BROOKLYN,N.Y. POLICE DEPT,986508,190249,"(40.688874254, -73.9918594329999)"',
 '4,638cd7b7,07/18/1972 11:00:00 PM,Tuesday,Jul,18,1972,23,7,19,2012,GRAND LARCENY OF MOTOR VEHICLE,FELONY,F,73,BROOKLYN,N.Y. POLICE DEPT,1005876,182440,"(40.6674141890001, -73.9220463899999)"',
 '5,6e410287,05/21/1987 12:01:00 AM,Thursday,May,21,1987,0,5,28,2009,GRAND LARCENY,FELONY,K,75,BROOKLYN,N.Y. POLICE DEPT,1017958,182266,"(40.6668988440001, -73.878495425)"']

In [18]:
dataWoHeader.map(lambda x: x.split(',')).take(5)

[['1',
  'f070032d',
  '09/06/1940 07:30:00 PM',
  'Friday',
  'Sep',
  '6',
  '1940',
  '19',
  '9',
  '7',
  '2010',
  'BURGLARY',
  'FELONY',
  'D',
  '66',
  'BROOKLYN',
  'N.Y. POLICE DEPT',
  '987478',
  '166141',
  '"(40.6227027620001',
  ' -73.9883732929999)"'],
 ['2',
  'c6245d4d',
  '12/14/1968 12:20:00 AM',
  'Saturday',
  'Dec',
  '14',
  '1968',
  '0',
  '12',
  '14',
  '2008',
  'GRAND LARCENY',
  'FELONY',
  'G',
  '28',
  'MANHATTAN',
  'N.Y. POLICE DEPT',
  '996470',
  '232106',
  '"(40.8037530600001',
  ' -73.955861904)"'],
 ['3',
  '716dbc6f',
  '10/30/1970 03:30:00 PM',
  'Friday',
  'Oct',
  '30',
  '1970',
  '15',
  '10',
  '31',
  '2008',
  'BURGLARY',
  'FELONY',
  'H',
  '84',
  'BROOKLYN',
  'N.Y. POLICE DEPT',
  '986508',
  '190249',
  '"(40.688874254',
  ' -73.9918594329999)"'],
 ['4',
  '638cd7b7',
  '07/18/1972 11:00:00 PM',
  'Tuesday',
  'Jul',
  '18',
  '1972',
  '23',
  '7',
  '19',
  '2012',
  'GRAND LARCENY OF MOTOR VEHICLE',
  'FELONY',
  'F',
  '73

In [20]:
import csv
from io import StringIO
from collections import namedtuple

In [21]:
fields = header.replace(" ","_").replace("/","_").split(',')
fields

['OBJECTID',
 'Identifier',
 'Occurrence_Date',
 'Day_of_Week',
 'Occurrence_Month',
 'Occurrence_Day',
 'Occurrence_Year',
 'Occurrence_Hour',
 'CompStat_Month',
 'CompStat_Day',
 'CompStat_Year',
 'Offense',
 'Offense_Classification',
 'Sector',
 'Precinct',
 'Borough',
 'Jurisdiction',
 'XCoordinate',
 'YCoordinate',
 'Location_1']

In [23]:
Crime = namedtuple('Crime',fields) # creating a class with each field as objects/properties in the class

__main__.Crime

In [27]:
def parse(row):
  reader = csv.reader(StringIO(row))
  row = next(reader)
  return Crime(*row)

In [64]:
crime = dataWoHeader.map(parse)

In [65]:
crime.first()

Crime(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)')

In [66]:
crime.first().Offense

'BURGLARY'

In [67]:
# take the offense value from each record and then count by the type of offense
crime.map(lambda x: x.Offense).countByValue()

defaultdict(int,
            {'BURGLARY': 40088,
             'GRAND LARCENY': 84508,
             'GRAND LARCENY OF MOTOR VEHICLE': 25773,
             'RAPE': 3317,
             'ROBBERY': 40280,
             'FELONY ASSAULT': 30989})

In [68]:
crime.map(lambda x: x.Occurrence_Year).countByValue()

defaultdict(int,
            {'1940': 1,
             '1968': 1,
             '1970': 1,
             '1972': 1,
             '1987': 4,
             '1990': 6,
             '1992': 5,
             '1994': 16,
             '1995': 24,
             '1996': 30,
             '1998': 72,
             '1999': 124,
             '2000': 277,
             '2001': 341,
             '2002': 366,
             '2003': 488,
             '2004': 686,
             '2005': 3256,
             '2006': 127321,
             '1910': 3,
             '1913': 4,
             '1945': 2,
             '1981': 1,
             '1985': 5,
             '1988': 3,
             '1991': 6,
             '1905': 2,
             '1971': 1,
             '1997': 38,
             '1914': 2,
             '1956': 1,
             '1989': 4,
             '1993': 16,
             '2015': 4,
             '1954': 1,
             '1982': 3,
             '1950': 1,
             '1959': 1,
             '1966': 7,
             '1980': 

In [71]:
crimesFiltered = crime.filter(lambda x: not(x.Occurrence_Year=='')).filter(lambda x: int(x.Occurrence_Year) >2005 and int(x.Occurrence_Year)<2015)

In [72]:
crimesFiltered.map(lambda x: x.Occurrence_Year).countByValue()

defaultdict(int, {'2006': 127321, '2007': 80833, '2008': 10965})

In [73]:
def extractCoords(location):
  location_lat = float(location[1:location.index(",")])
  location_lon = float(location[location.index(",")+1:-1])
  return (location_lat,location_lon)

In [74]:
crimesFiltered.map(lambda x: extractCoords(x.Location_1))\
              .reduce(lambda x,y:(min(x[0],y[0]),min(x[1],y[1])))

(40.112709974, -77.519206334)

In [75]:
crimesFiltered.map(lambda x: extractCoords(x.Location_1))\
              .reduce(lambda x,y:(max(x[0],y[0]),max(x[1],y[1])))

(40.912723396, -73.700716685)

In [76]:
# filtering records that are in the boundaries of new york city co-ordinates
crimeFinal = crimesFiltered.filter(lambda x: extractCoords(x.Location_1)[0]>=40.1 and \
                                   extractCoords(x.Location_1)[0]<= 40.95 and \
                                   extractCoords(x.Location_1)[1] >= -77.4 and \
                                   extractCoords(x.Location_1)[1] <= 73.2 )


In [77]:
crimeFinal.count()

219118

In [78]:
crimeFinal.map(lambda x: x.Occurrence_Year).countByValue()

defaultdict(int, {'2006': 127321, '2007': 80832, '2008': 10965})

In [79]:
crimeFinal.filter(lambda x: x.Offense=='BURGLARY')\
          .map(lambda x: x.Occurrence_Year)\
          .countByValue()

defaultdict(int, {'2006': 23069, '2007': 14736, '2008': 1994})

In [84]:
!pip install geopandas
import plotly.express as px
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 29.3 MB/s 
     |████████████████████████████████| 16.6 MB 56.6 MB/s 
     |████████████████████████████████| 7.8 MB 54.8 MB/s 


In [82]:
b_lats = crimeFinal.filter(lambda x: x.Offense=='BURGLARY' and x.Occurrence_Year=='2008')\
                    .map(lambda x: extractCoords(x.Location_1)[0])\
                    .collect()

b_lons = crimeFinal.filter(lambda x: x.Offense=='BURGLARY' and x.Occurrence_Year=='2008')\
                    .map(lambda x: extractCoords(x.Location_1)[1])\
                    .collect()



In [89]:
import plotly.express as px
import geopandas as gpd
import plotly.graph_objects as go

px.set_mapbox_access_token(open(".mapbox_token").read())

fig = go.Figure(data=go.Scattergeo(
        lon = b_lons,
        lat = b_lats,
        ))
fig.update_geos(fitbounds="locations")
fig.update_layout(
        title = 'Crimes in NYC',
    )
fig.show()